In [1]:
import requests
import json
import pandas as pd
import datetime
import math
import statistics as st
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Button, Layout

## Инструмент для определения компаний-агрессоров по отраслям

Представляю вашему вниманию инструмент для выявления и анализа компаний, которые активно растут и предлагают зарплаты, значимо выше рыночных. Все предлагаемые выводы основаны на открытых данных **hh.ru**.

### Описание интерфейса:

- Уровень 1. Здесь из списка, взятого с hh.ru, вы можете выбрать первый уровень отрасли (н-р, "Нефть и газ")


- Уровень 2. Здесь формируется список отраслей второго уровня, которые соответствуют выбранному ранее первому. Здесь можно выбрать несколько значений сразу. Чтобы это сделать, зажмите Ctrl/cmd и кликайте по нужным элементам. Если вы хотите получить анализ всей отрасли первого уровня, не выбирайте ничего. (Н-р, "Добыча нефти")


- Слайдер с минимальной зарплатой. Вы можете выставить зарплату, чтобы отбросить компании с медианной зарплатой ниже.


- Слайдер с минимальным числом вакансий. Вы можете выставить число, чтобы отбросить компании с числом активных вакансий меньше.


- Кнопка. Когда все параметры выставлены, вы можете нажать на кнопку, чтобы запустить процесс поиска и анализа.

**Дополнительно:**

После нажатия на кнопку запустится процесс поиска и анализа - это может занять некоторое время (до 10-15 минут, если не выбрана отрасль второго уровня, а отрасль первого уровня довольно обширна), так как именно здесь "штурмуется" API hh.ru. Для удобства появится бегунок с загрузкой, который подскажет, стоит ли пойти налить чай, или можно и подождать полминуты.

Значения слайдеров можно настроить до запуска поиска и анализа, в процессе они будут заморожены. Однако когда на экране появятся аналитические выводы, можно будет поменять значения, чтобы подкорректировать результат.

В результате анализа может быть сказано, что компаний-агрессоров не найдено. Это означает, либо что по заданным параметрам не найдено компаний вовсе, либо что найденные компании не отличаются "агрессивностью" (Н-р, задана высокая минимальная медианная зарплата, но компании отвечающие требованиям, сильно уступают по другим параметрам).

Краткие пометки к технической части инструментов вы найдете в коде. Обоснование методологии приведено в блокноте после инструмента.

*Приятного теста!*

In [2]:
class HHParser: #сам класс
    def __init__(self):
        self.loading_output = widgets.Output() #аутпут для бегунка прелоадера
        self.loading = widgets.FloatProgress( #настройки прелоадера
            value=0,
            min=0,
            max=10.0,
            description='Loading:',
            bar_style='success',
            style={'bar_color': '#00FF00'},
            orientation='horizontal'
        )
        
        display(self.loading_output) #включение прелоадера для первичной загрузки отраслей
        with self.loading_output:
            display(self.loading)
        self.loading.value = 2 #апдейт лоадера, он будет встречаться много где
        self.get_industries() #функция, которая берет с hh.ru текущий список отраслей
        self.loading.value = 4
        self.get_currencies() #функция, которая с открытого api берет нынешние курсы валют (понадобятся далее)
        self.loading.value = 6
        self.button = widgets.Button(description = 'Найти агрессоров') #кнопка для запуска
        
        self.level_1_widget = widgets.Dropdown( #виджеты для выбора отраслей
            value='Строительство, недвижимость, эксплуатация, проектирование',
            placeholder='Choose Someone',
            options=list(self.industries_df['name_first'].unique()),
            description='Уровень 1:',
            ensure_option=True,
            disabled=False,
            layout=Layout(width='70%')
        )
        self.loading.value = 8
        self.level_2_widget = widgets.SelectMultiple(
            placeholder='Choose Someone',
            options=list(self.industries_df['name_second'][self.industries_df['name_first'] == 'Строительство, недвижимость, эксплуатация, проектирование']),
            description='Уровень 2:',
            ensure_option=True,
            disabled=False,
            layout=Layout(width='70%'),
            rows = 20
        )
        
        layout = widgets.Layout(width='auto', height='60px')
        self.salaries_slider = widgets.FloatSlider( #настройка слайдеров
            value=0,
            min=0,
            max=500000,
            step=5000,
            description='Минимальная медианная зарплата в компании для признания ее агрессором:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.0f',
            layout = layout,
            style={'description_width': 'initial', 'width': '800px'}
        )
            
        self.vacancies_slider = widgets.FloatSlider(
            value=0,
            min=0,
            max=500,
            step=2,
            description='Минимальное количество открытых в компании вакансий для признания ее агрессором:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.0f',
            layout = layout,
            style={'description_width': 'initial', 'width': '800px'}
        )
        
        self.output = widgets.Output() #основной аутпут, в котором будут результаты анализа
        self.salaries_slider.observe(self.on_salaries_slider_change) #настройки поведения виджетов, финкции ниже
        self.vacancies_slider.observe(self.on_vacancies_slider_change)
        self.level_1_widget.observe(self.on_change)
        self.button.on_click(self.on_click)
        self.loading.value = 10
        self.loading_output.clear_output() #сброс лоадера
        display(self.level_1_widget)
        display(self.level_2_widget)
        display(self.button)
        display(self.vacancies_slider)
        display(self.salaries_slider)
        display(self.output)
        return None
    
    def on_change(self, change): #при изменении отрасли первого уровня обновляется список подотраслей
        if change['type'] == 'change' and change['name'] == 'value':
            self.level_2_widget.options = list(self.industries_df['name_second'][self.industries_df['name_first'] == change['new']])
            
            
    def on_click(self, click): #запуск процесса анализа
        self.output.clear_output()
        self.vacancies_slider.disabled = True
        self.salaries_slider.disabled = True
        value = self.level_2_widget.value
        if len(value) == 0: #проверяем, выбрана ли отрасль второго уровня
            industries = list(self.industries_df[self.industries_df['name_first'] == self.level_1_widget.value]['id_first'])[0]
        else:
            industries = [self.industries_df[self.industries_df['name_second'] == industry_name]['id_second'].iloc[0] for industry_name in list(value)]
        self.find_employers(industries) #основная функция для запуска
    
    def on_salaries_slider_change(self, change): #при изменении значения слайдера (нужна, чтобы вносить корректировки в уже выданный результат)
        if change['type'] == 'change' and change['name'] == 'value':
            self.output.clear_output()
            try:
                self.display_result(minimal_salary = change['new'], minimal_vacancies = self.vacancies_slider.value)
            except Exception as err:
                pass
        
    def on_vacancies_slider_change(self, change): #аналогична предыдущей, но для слайдера с количеством вакансий
        if change['type'] == 'change' and change['name'] == 'value':
            self.output.clear_output()
            try:
                self.display_result(minimal_vacancies = change['new'], minimal_salary = self.salaries_slider.value)
            except Exception as err:
                pass
            
    def display_result(self, minimal_salary, minimal_vacancies): #функция для вывода результата
        self.output.clear_output()
        #фильтрация по коэффициенту агрессивности и заданным на слайдерах параметрах
        agressive_employers_idx = list(self.employers_df[(self.employers_df['agressive_coefficient'] >= self.threshold) & (self.employers_df['median_salary'] >= minimal_salary) & (self.employers_df['vacancies_number_real'] >= minimal_vacancies)].index)
        if len(agressive_employers_idx) == 0:
            with self.output:
                print('\033[1mНе удалось найти агрессоров, отвечающих заданным параметрам. Возможно, стоит снизить пороговые значения.\033[0m')
        else:
            with self.output:
                for index in agressive_employers_idx: #выводы и графики
                    employer = self.employers_df.iloc[index]
                    print(f"\033[7m{employer['name']}\033[0m")
                    print(f'Медианная зарплата в {employer["name"]} выше аналогичного показателя в отрасли на \033[1m{round(employer["percentage_higher"] * 100, 2)}%\033[0m и равна \033[1m{employer["median_salary"]} руб.\033[0m')
                    print(f'Открытые вакансии {employer["name"]} составляют \033[1m{round(employer["vacancies_share"] * 100, 2)}%\033[0m от всех вакансий отрасли.\nВсего: \033[1m{employer["vacancies_number_real"]}\033[0m')
                    print(f'В \033[1m{round(employer["with_salary"]*100, 2)}%\033[0m вакансий указана заработная плата. Из них \033[1m{round(employer["high_salaries_share"]*100, 2)}\033[0m% выше медианной в отрасли.')

                    plt.figure(figsize = (15, 5))
                    plt.hist(self.all_salaries, bins = 50)
                    plt.hist(employer['salaries'])
                    plt.legend(['Отрасль', employer['name']])
                    plt.title('Распределение зарплат в отрасли vs в компании')

                    plt.figure(figsize = (15, 5))
                    plt.bar(x = [i for i in range(len(self.employers_df))], height = self.employers_df['vacancies_number_real'])
                    plt.hist(x = '', height = employer['vacancies_number_real'], color = 'orange')
                    plt.legend(['Отрасль', employer['name']])
                    plt.title('Количество вакансий в других компаниях и в этой')
                    plt.show()
            
    
    def get_industries(self): #функция, которая берет с hh.ru список отраслей
        url = 'https://api.hh.ru/industries'
        response = requests.get(url)
        raw_df = pd.DataFrame(response.json())
        response.close()
        self.industries_df = pd.DataFrame(columns = ['id_first', 'id_second', 'name_first', 'name_second'])
        for i in range(len(raw_df)): #парсим полученный ответ и фиксируем
            row = raw_df.iloc[i]
            id_first = row['id']
            name_first = row['name']
            second_level = row['industries']
            for item in second_level:
                id_second = item['id']
                name_second = item['name']
                new_row = pd.Series({'id_first': id_first,
                                     'name_first': name_first,
                                     'id_second': id_second,
                                     'name_second': name_second})
                self.industries_df = self.industries_df.append(new_row, ignore_index = True)
    
    def get_currencies(self): #функция, которая берет текущие курсы рубля
        url = 'https://api.exchangerate-api.com/v4/latest/RUB'
        response = requests.get(url)
        self.currencies_rates = response.json()['rates']
                
    def find_employers(self, industries): #основная функция, которая работает с hh.ru
        self.industries_ids = industries
        self.loading.value = 0.1
        with self.loading_output:
            display(self.loading) #загрузка прелоадера
        
        url = 'https://api.hh.ru/vacancies/'
        params = {
            'industry' : industries,
            'per_page' : 100,
            'area' : [1, 2], #смотрим только мск/спб
            'page' : 1,
            'period' : 30
            
        }
        self.loading.value += 0.1
        
        response = requests.get(url, params = params)
        self.loading.value += 0.1
        items_found = response.json()['found']
        response.close()
        self.loading.value += 0.2
        salaries_mult = {0: 1, 1: 0.87}
        n_pages = math.ceil(min(2000, items_found) / 100) #определяем, сколько страниц смотреть - максимум, доступный на hh.ru или найдено меньше
        loading_coefficient = 4.5 / (n_pages * 100) #коэффициент для увеличения лоадера на каждом шаге цикла
        employers = []
        employers_ids = []
        #ниже только предварительный поиск. цель - найти работодателей, которые разместили вакансии на hh. Есть риск, что мы найдем не всех (hh api не дает просматривать больше 2000 вакансий), но, поскольку нас интересуют только свежие, вероятность ошибки минимальна
        for page in range(n_pages): #просматриваем каждую страницу, чтобы найти работодателей
            params = {
                'industry' : industries,
                'per_page' : 100,
                'area' : [1, 2],
                'page' : page,
                'period' : 30 #вакансии опубликованные только за последние 30 дней
                }
            response = requests.get(url, params = params)
            items = response.json()['items']
            response.close()
            for item in items:
                self.loading.value += loading_coefficient
                if not item['archived']: #проверяем, не закрыта ли вакансия
                    employer = item['employer']['name']
                    try:
                        employer_id = item['employer']['id']
                    except:
                        employer_id = None
                    if employer_id and employer_id not in employers_ids:
                        employers_ids.append(employer_id) #запоминаем id и имя компании
                        employers.append(employer)
        self.employers_df = pd.DataFrame(columns = ['name', 'id'])
        self.employers_df['name'] = employers
        self.employers_df['id'] = employers_ids #объединяем найденные данные в таблицу
        self.employers_number = len(self.employers_df['id'])
        
        employers_data = self.employers_df['id'].apply(self.learn_employer) #для каждого id ищем данные, т.е. смотрим вакансии по конкретным компаниям
        
        self.employers_df['vacancies_number_real'] = [item[0] for item in employers_data] #полученные данные добавляем в таблицу
        self.employers_df['vacancies_number_observed'] = [item[1] for item in employers_data]
        self.employers_df['salaries'] = [item[2] for item in employers_data]
        self.employers_df['median_salary'] = [item[3] for item in employers_data]
        self.employers_df['with_salary'] = [item[4] for item in employers_data]
        self.loading_output.clear_output() #сброс прелоадера - все долгие шаги уже прошли, осталось менее секунды
        
        self.all_salaries = self.concat_lists(list(self.employers_df['salaries']))
        self.median_salary = st.median(self.all_salaries)
                            
        self.employers_df['vacancies_share'] = self.employers_df['vacancies_number_observed'] / sum(self.employers_df['vacancies_number_observed']) #расчет доли "рынка вакансий"
        self.employers_df['high_salaries_share'] = self.employers_df['salaries'].apply(self.high_salaries_share) #расчет доли высоких зарплат в компании
        self.employers_df['relative_to_industry'] = self.employers_df['median_salary'] / self.median_salary #отношение медианных зарплат рвботодателя и всей отрасли
        self.employers_df['is_higher'] = (self.employers_df['relative_to_industry'] > 1).astype('int') #проверка, выше ли медианная зарплата
        self.employers_df['percentage_higher'] = self.employers_df['relative_to_industry'] - 1 #технический столбец, потом понадобится для вывода
        
        self.employers_df['agressive_coefficient'] = self.employers_df['vacancies_share'] * self.employers_df['high_salaries_share'] * self.employers_df['is_higher'] * self.employers_df['relative_to_industry'] * self.employers_df['with_salary'] #рассчет итогового показателя, описан в методологии
        
        self.threshold = np.percentile(self.employers_df['agressive_coefficient'], 95) #принимаем, что агрессоры находятся за пределами 95 перцентиля (2 сигмы, если распредление нормальное)
        self.display_result(minimal_vacancies = self.vacancies_slider.value, minimal_salary = self.salaries_slider.value) #выводим результат
        self.vacancies_slider.disabled = False #активируем слайдеры
        self.salaries_slider.disabled = False
    
    def high_salaries_share(self, salaries): #вспомогательная функция для расчета доли высоких зарплат
        if salaries == []:
            return 0
        else:
            high_salaries_num = [1 for salary in salaries if salary > self.median_salary]
            return sum(high_salaries_num) / len(salaries)
        
    def concat_lists(self, lists): #вспомогательная функция для сцепки списков
        result = []
        for l in lists:
            result += l
        return result
                            
            
    def learn_employer(self, employer_id): #изучение вакансий одного работодателя
        params = {
            'industry' : self.industries_ids,
            'employer_id' : employer_id,
            'per_page' : 100,
            'area' : [1, 2],
            'page' : 1,
            'period' : 30
        }
        salaries_mult = {0: 1, 1: 0.87} #все переводим в "на руки"
        response = requests.get('https://api.hh.ru/vacancies', params = params)
        vacancies_number_real = response.json()['found']
        vacancies_number_observed = min(2000, vacancies_number_real)
        response.close()
        pages =  math.ceil(vacancies_number_observed / 100)
        loading_coefficient = 5 / vacancies_number_observed / self.employers_number #коэффициент для движения прелоадера на каждом шаге цикла
        salaries = []
        for page in range(pages):
            params['page'] = page
            response = requests.get('https://api.hh.ru/vacancies', params = params) 
            items = response.json()['items']
            response.close()
            for item in items:
                self.loading.value += loading_coefficient
                salary = item['salary'] 
                if salary: #проверяем, указана ли зарплата
                    try:
                        salary_from = salary['from'] #парсим от-до
                        salary_to = salary['to']
                        if not salary_from:
                            salary_from = salary_to
                        elif not salary_to:
                            salary_to = salary_from
                        salary_from = salary_from * salaries_mult[int(bool(salary['gross']))] #переводим в "на руки"
                        salary_to = salary_to * salaries_mult[int(bool(salary['gross']))]
                        if salary['currency'] != 'RUR': #переводим, если зп указана в другой валюте
                            currency_rate = self.currencies_rates[salary['currency']]
                            salary_from = salary_from / currency_rate
                            salary_to = salary_to / currency_rate
                    except Exception as err:
                        print(err)
                        salary_from = 0
                        salary_to = 0
                else:
                    salary_from = 0
                    salary_to = 0
                salary = (salary_from + salary_to) / 2
                if salary:
                    salaries.append(salary)
        without_salary = vacancies_number_observed - len(salaries)
        if len(salaries):
            median_salary = st.median(salaries)
        else:
            median_salary = 0
        try:
            with_salary =  len(salaries) / vacancies_number_observed
        except:
            with_salary = 0
        # в итоге реальное кол-во вакансий, изученное кол-во вакансий (меньше, если в сумме их больше 2000), все найденные зарплаты, доля вакансий с указанной зп
        return (vacancies_number_real, vacancies_number_observed, salaries, median_salary, with_salary)

In [4]:
#запустите эту ячейку
#запись в переменную, чтобы, если захочется, можно было посмотреть на датасет
hh = HHParser()

Output()

Dropdown(description='Уровень 1:', index=5, layout=Layout(width='70%'), options=('Перевозки, логистика, склад,…

SelectMultiple(description='Уровень 2:', layout=Layout(width='70%'), options=('Строительство коммерческих объе…

Button(description='Найти агрессоров', style=ButtonStyle())

FloatSlider(value=0.0, continuous_update=False, description='Минимальное количество открытых в компании ваканс…

FloatSlider(value=0.0, continuous_update=False, description='Минимальная медианная зарплата в компании для при…

Output()

## Обоснование методологии

#### Основные коэффициенты
- `vacancies_share` - доля опубликованных вакансий, которая принадлежит компании.
- `with_salary` - доля вакансий компании с указанной зарплатой.
- `high_salaries_share` - доля зарплат в вакансиях компании, которые выше медианной в отрасли.
- `relative_to_industry` - отношение медианной зарплаты в компании медианной зарплаты в отрасли.
- `is_higher` - бинарный коэффициент: выше ли медианная зарплата в компании медианной в отрасли.

#### Итоговый коэффициент

Итоговый коэффициент равен произведению перечисленных выше показателей. Может показаться, что "намудрил". На самом же деле, итоговый показатель, во-первых, максимально "человечный" и, во-вторых, основан на школьной теории вероятности. 

Все по порядку:


Когда мы задаемся вопросом "Как определить компанию-агрессора?", мне кажется, стоит пойти с обратной стороны: посмотреть глазами человека, которого мы рискуем потерять, на рынок труда. Он заходит на hh.ru и листает вакансии. И вот тут мы можем задать себе правильный вопрос "Что будет делать компания-агрессор, чтобы привлечь к себе внимание?". Ей нужно привлечь внимание, а именно повысить вероятность того, что человек откликнется на вакансию.

Тут переходим к коэффициенту:

Первый - вероятность того, что случайно выбранная вакансия принадлежит компании.

Второй - вероятность того, что у этой вакансии указана зарплата.

Третий - вероятность того, что эта зарплата "выше рынка" (т.е. привлекательна для соискателя).

Из этих трех параметров состоит "тело" коэффициента агрессивности. Итого этим телом мы получаем ответ на вопрос: 
###### Какова вероятность того, что случайно выбранная вакансия будет принадлежать компании и у нее будет указана зарплата выше рыночной?

Иными словами, насколько вероятно, что компания ведет агрессивный подбор, активно привлекая блуждающих по hh.ru чужих сотрудников.

Оставшиеся два показателя:

Отношение медианной зарплаты компании к отраслевому показателю. Иными словами "А насколько больше?". Это тоже важно. Сам факт наличия повышенных зарплат ничего не говорит, если они выше рынка на полкопейки. Можно сказать, что это поправочная составляющая итогового коэффициента.

Бинарный коэффициент просто отсекает обнуляет все вышесказанное. Да, возможно некоторые вакансии компании и отличаются повышенными зарплатами, но если, в целом, они ниже отраслевой медианы, то говорит об агрессивном наборе не приходится. Эту составляющую можно было бы убрать, если бы мы, помимо отрасли, делали разбивку по грейдам/отдельным профессиям, но в рамках этой работы такая деталезация не предусмотрена.

###### P.s.
На самом деле, мне кажется более рациональным "смотреть" по отдельным профессиям, так как может сложиться ситуация, когда компания агрессивно набирает сотрудников именно на линейные роли, но в разрезе отрасли этого не будет видно из-за того, что повышенные зарплаты низких грейдов все равно ниже, чем обычные зарплаты высоких грейдов.